In [32]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import twokenize
from nltk.tokenize import TweetTokenizer
from FeatureFunctions import getfeatures
import sklearn
import csv


### Import the classifier
import sys
sys.path.insert(0, 'libsvm')

from svmutil import *

In [33]:
# whichtask = input("Which task to you want to do (A/B): ")
whichtask = "A"

In [35]:
### Reading train and test files

datafile = "datasets/train/SemEval2018-T3-train-task"+whichtask+"_emoji.txt"
# datafile2 = "datasets/train/SemEval2018-T3-train-taskA_emoji_ironyHashtags.txt"
trainingdata = pd.read_csv(datafile, delimiter = "\t",  quoting=csv.QUOTE_NONE, header=0)
trainingdata = trainingdata[['Label','Tweet text']]

testfile = 'datasets/goldtest_Task'+whichtask+'/SemEval2018-T3_gold_test_task'+whichtask+'_emoji.txt'
testdata = pd.read_csv(testfile, sep="\t",  quoting=csv.QUOTE_NONE, header=0)
testdata = testdata[['Label','Tweet text']]


In [36]:
### Get lexical features
# training_features
lexical_training_features, unicount_vect, bicount_vect, tricount_vect, fourcount_vect = getfeatures.getlexical(trainingdata, 'Tweet text')
x_small = lexical_training_features[['PunctuationFlood', 'CharFlood', 'CapitalizedCount', 'HashtagCount', 'Hashtag2WordRatio', 'TweetCharLength', 'TweetWordLength', 'EmojiCount', 'FinalPunctuation']].values
x_lexical = np.array(lexical_training_features.apply(lambda row: sum([row['CharFourgramVector'], row['CharTrigramVector'],row['BigramVector']], []), axis=1).values.tolist())
x_lexical = np.hstack((x_small, x_lexical))

# train_bow = lexical_training_features['UnigramVector'].values.tolist()

# test_features
lexical_test_features, unicount_vect, bicount_vect, tricount_vect, fourcount_vect = getfeatures.getlexical(testdata, 'Tweet text', unicount_vect, bicount_vect, tricount_vect, fourcount_vect)
test_x_small = lexical_test_features[['PunctuationFlood', 'CharFlood', 'CapitalizedCount', 'HashtagCount', 'Hashtag2WordRatio', 'TweetCharLength', 'TweetWordLength', 'EmojiCount', 'FinalPunctuation']].values
test_lexical_x = np.array(lexical_test_features.apply(lambda row: sum([row['CharFourgramVector'], row['CharTrigramVector'],row['BigramVector']], []), axis=1).values.tolist())
test_lexical_x = np.hstack((test_x_small, test_lexical_x))

train_bow = np.array(lexical_training_features['UnigramVector'].values.tolist())
test_bow = np.array(lexical_test_features['UnigramVector'].values.tolist())

lexical_training_features = []
lexical_test_features = []




In [64]:
from importlib import reload  # Python 3.4+ only.
reload(getfeatures)

### Get sentiment features
train_sentiment_x = getfeatures.getaffinfeats(trainingdata['Tweet text'])
test_sentiment_x = getfeatures.getaffinfeats(testdata['Tweet text'])


In [65]:
### Creating 50-50% balance
## Train

### Count the amount of samples
amount_nonirony_train = sum(trainingdata["Label"] == 0)
amount_irony_train = sum(trainingdata["Label"] > 0)
amount_train_amount = min(amount_nonirony_train, amount_irony_train)

### Sample indices
train_nonirony_index = trainingdata[trainingdata["Label"] == 0].index.to_series()
train_irony_index = trainingdata[trainingdata["Label"] > 0].index.to_series()

train_nonirony_index_samples = train_nonirony_index.sample(amount_train_amount).tolist()
train_irony_index_samples = train_irony_index.sample(amount_train_amount).tolist()

## Test
amount_nonirony_test = sum(testdata["Label"] == 0)
amount_irony_test = sum(testdata["Label"] > 0)
amount_test_amount = min(amount_nonirony_test, amount_irony_test)

### Sample indices
test_nonirony_index = testdata[testdata["Label"] == 0].index.to_series()
test_irony_index = testdata[testdata["Label"] > 0].index.to_series()

test_nonirony_index_samples = test_nonirony_index.sample(amount_test_amount).tolist()
test_irony_index_samples = test_irony_index.sample(amount_test_amount).tolist()



In [82]:
### Selecting the actual training samples (Do some crossover here )

resulting_train_index = train_nonirony_index_samples +train_irony_index_samples
trainingdata_result = trainingdata.loc[resulting_train_index]

resulting_test_index = test_nonirony_index_samples +test_irony_index_samples
testdata_result = testdata.loc[resulting_test_index]

### Combining features
quitting = 1

while(quitting):
    whichtype = int(input("Which type to test: "))
    
    quitting = int(input("Quit afterwards? (0 to quit): "))

    final_train_x = []
    final_test_x = []

    ## BoW
    if whichtype == 1:
        final_train_x = train_bow[resulting_train_index]
        final_test_x = test_bow[resulting_test_index]
    ## Lexical
    if whichtype == 2:
        final_train_x = np.hstack((train_bow[resulting_train_index], x_lexical[resulting_train_index]))
        final_test_x = np.hstack((test_bow[resulting_test_index], test_lexical_x[resulting_test_index]))
    ## Sentiment
    if whichtype == 4:
        final_train_x = train_sentiment_x[resulting_train_index]
        final_test_x = test_sentiment_x[resulting_test_index]
    ## Combined
    if whichtype == 6:
        final_train_x = np.hstack((train_bow[resulting_train_index], train_sentiment_x[resulting_train_index], x_lexical[resulting_train_index]))
        final_test_x = np.hstack((test_bow[resulting_test_index], test_sentiment_x[resulting_test_index], test_lexical_x[resulting_test_index]))


    print("Training:")
    print(final_train_x.shape)
    print()
    ### Train and get train error
    y = trainingdata_result['Label'].tolist()
    prob  = svm_problem(y, final_train_x)
    param = svm_parameter('-t 2 -c 8 -g ' + str(2**-11))
    m = svm_train(prob, param)
    p_label, p_acc, p_val = svm_predict(y, final_train_x, m)
    ACC, MSE, SCC = evaluations(y, p_label)

    print("Testing:")
    print(final_test_x.shape)
    print()


    ### Get the test
    test_y = testdata_result['Label'].tolist()

    test_p_label, test_p_acc, test_p_val = svm_predict(test_y, final_test_x, m)
    test_ACC, test_MSE, test_SCC = evaluations(test_y, test_p_label)
    print(sklearn.metrics.classification_report(test_y, test_p_label, digits=4))

    if whichtask == "A":
        p, r, f = helper.precision_recall_fscore(test_y, test_p_label, beta=1, labels=[0,1], pos_label=1)
    elif whichtask == "B":
        p, r, f = helper.precision_recall_fscore(test_y, test_p_label, beta=1, labels=[0,1,2,3])

    print("Precision: " + str(p))
    print("Recall: " + str(r))
    print("F1-score: " + str(f))

    y_actu = pd.Series(test_y, name='Actual')
    y_pred = pd.Series(test_p_label, name='Predicted')
    df_confusion = pd.crosstab(y_actu, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

    print()
    print()
    print(df_confusion)

Which type to test: 6
Quit afterwards? (0 to quit): 0
Training:
(3822, 63417)

Accuracy = 90.1884% (3447/3822) (classification)
Testing:
(622, 63417)

Accuracy = 67.0418% (417/622) (classification)
              precision    recall  f1-score   support

           0     0.6541    0.7235    0.6870       311
           1     0.6906    0.6174    0.6520       311

   micro avg     0.6704    0.6704    0.6704       622
   macro avg     0.6724    0.6704    0.6695       622
weighted avg     0.6724    0.6704    0.6695       622

Precision: 0.6906474820143885
Recall: 0.617363344051447
F1-score: 0.6519524617996604


Predicted  0.0  1.0  All
Actual                  
0          225   86  311
1          119  192  311
All        344  278  622


In [84]:
print(amount_nonirony_test)
print(amount_irony_test)

473
311
